In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

Do not ever put API's in your repositories. This one regenerates after 24 hours and doesn't have any sensitive information behind it. But do as I say, not as I do.

In [6]:
myApi = 'RGAPI-9102b880-fd87-40e3-bc9c-25239d70a4c4'
lol_watcher = LolWatcher(myApi)

region = 'na1'


In [18]:
def get_puuid(name):
    """
    Input is summoner name. Outputs players puuid.
    """
    return lol_watcher.summoner.by_name(region, name)['puuid']
def get_matchlist(name):
    """
    Input is name. Output is list of matches.
    """
    return lol_watcher.match.matchlist_by_puuid(region, get_puuid(name), count=100, type='ranked')


In [9]:
summoners = ['Zealfire', 'SmurfHD', 'DumbAmerican', 'About 10 Inches', 'Exprience', 'Leaders1', 
             'Kehlee', 'KybitOW', 'DodoMuncher', 'Mango Bleach']

In [19]:
get_matchlist('Zealfire')

['NA1_4348171334',
 'NA1_4289259129',
 'NA1_4287742329',
 'NA1_4287694201',
 'NA1_4287648113',
 'NA1_4286956599',
 'NA1_4286785334',
 'NA1_4286667294',
 'NA1_4285375587',
 'NA1_4285360882',
 'NA1_4285334783',
 'NA1_4285247595',
 'NA1_4284641450',
 'NA1_4284573748',
 'NA1_4282617016',
 'NA1_4282651208',
 'NA1_4282568419',
 'NA1_4282560428',
 'NA1_4279275380',
 'NA1_4279290874',
 'NA1_4272162489',
 'NA1_4270474097',
 'NA1_4270373912',
 'NA1_4270325535',
 'NA1_4270310445',
 'NA1_4269678873',
 'NA1_4269623819',
 'NA1_4265939472',
 'NA1_4265052963',
 'NA1_4265007120',
 'NA1_4265011466',
 'NA1_4264596019',
 'NA1_4264558755',
 'NA1_4263428731',
 'NA1_4262510186',
 'NA1_4262424973',
 'NA1_4259122335',
 'NA1_4259076750',
 'NA1_4258616238',
 'NA1_4257716745',
 'NA1_4251471488',
 'NA1_4248601449',
 'NA1_4248509505',
 'NA1_4247935338',
 'NA1_4243646495',
 'NA1_4242625676',
 'NA1_4242377491',
 'NA1_4242051531',
 'NA1_4241728549',
 'NA1_4241732889',
 'NA1_4240856758',
 'NA1_4240829863',
 'NA1_423986

In [26]:
pd.DataFrame([lol_watcher.match.by_id(my_region, 'NA1_4348171334')['info']['participants'][0]])

,assists,baronKills,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,...,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,6,0,6,"{'12AssistStreakCount': 0, 'abilityUses': 93, ...",13245,15,133,Quinn,0,5,...,1062,1,1,3,0,18,1,0,7,True


In [5]:
data = lol_watcher.league.entries(my_region, queue='RANKED_SOLO_5x5', tier='GOLD', division='II', page=500)
len(data)

0

In [24]:
lol_watcher.match.by_id(my_region, 'NA1_4348171334')['info']

{'gameCreation': 1655686497000,
 'gameDuration': 1449,
 'gameEndTimestamp': 1655687977149,
 'gameId': 4348171334,
 'gameMode': 'CLASSIC',
 'gameName': 'teambuilder-match-4348171334',
 'gameStartTimestamp': 1655686527674,
 'gameType': 'MATCHED_GAME',
 'gameVersion': '12.11.446.9344',
 'mapId': 11,
 'participants': [{'assists': 6,
   'baronKills': 0,
   'bountyLevel': 6,
   'challenges': {'12AssistStreakCount': 0,
    'abilityUses': 93,
    'acesBefore15Minutes': 0,
    'alliedJungleMonsterKills': 4,
    'baronBuffGoldAdvantageOverThreshold': 1,
    'baronTakedowns': 1,
    'blastConeOppositeOpponentCount': 0,
    'bountyGold': 0,
    'buffsStolen': 0,
    'completeSupportQuestInTime': 0,
    'controlWardTimeCoverageInRiverOrEnemyHalf': 0.10293728200029126,
    'controlWardsPlaced': 1,
    'damagePerMinute': 893.1887325443008,
    'damageTakenOnTeamPercentage': 0.15137983692527165,
    'dancedWithRiftHerald': 0,
    'deathsByEnemyChamps': 4,
    'dodgeSkillShotsSmallWindow': 28,
    'dou

In [ ]:
i